In [31]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import joblib
from sklearn.metrics import classification_report, confusion_matrix

In [37]:
test_class_folders = [
(r"E:\Jain Irrigation\data\A\test_cropped", 1),
(r"E:\Jain Irrigation\data\BC\test_cropped", 2),
(r"E:\Jain Irrigation\data\D\test_cropped", 3)
]

In [38]:
def load_test_images_with_padding(folder_label_pairs, image_size=(512,512), pad_color=(255,255,255)):
    data = []
    filenames = []
    for folder, label in folder_label_pairs:
        image_files = [f for f in os.listdir(folder)
                       if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        for filename in tqdm(image_files, desc=f"Processing {os.path.basename(folder)}"):
            path = os.path.join(folder, filename)
            try:
                img = Image.open(path).convert("RGB")
                img.thumbnail(image_size, Image.Resampling.LANCZOS)

                background = Image.new("RGB", image_size, pad_color)
                offset = ((image_size[0] - img.width) // 2,
                          (image_size[1] - img.height) // 2)
                background.paste(img, offset)

                arr = np.array(background) / 255.0
                flat = arr.flatten()

                data.append(flat)
                filenames.append((filename, label))
            except Exception as e:
                print(f"⚠️ Skipped {filename}: {e}")
    X = np.array(data)
    y = np.array([label for _, label in filenames])
    file_names = [name for name, _ in filenames]
    return X, y, file_names


In [41]:
svm_model = joblib.load("svm_model.pkl")
log_reg_model = joblib.load("softmax_1.pkl")

print("✅ Models loaded successfully!")

✅ Models loaded successfully!


In [42]:
X_test, y_test, file_names = load_test_images_with_padding(test_class_folders)
print("Test shape:", X_test.shape, "Labels:", len(y_test))

Processing test_cropped: 100%|█████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 84.94it/s]


Test shape: (97, 786432) Labels: 97


In [43]:
# --- Predict with Logistic Regression ---
log_probs = log_reg_model.predict_proba(X_test)
log_preds = np.argmax(log_probs, axis=1)
log_labels = log_reg_model.classes_
log_acc = (log_labels[log_preds] == y_test).mean()

print(f"\nLogistic Regression Accuracy on new images: {log_acc:.3f}")
for i, name in enumerate(file_names):
    print(f"{name:<25} | True: {y_test[i]:<3} | Pred: {log_labels[log_preds[i]]:<3} | Prob: {log_probs[i][log_preds[i]]:.4f}")



Logistic Regression Accuracy on new images: 0.639
IMG_20250920_114308655.jpg | True: 1   | Pred: 1   | Prob: 1.0000
IMG_20250920_114321040.jpg | True: 1   | Pred: 1   | Prob: 0.9998
IMG_20250920_114337247.jpg | True: 1   | Pred: 1   | Prob: 1.0000
IMG_20250920_114349302.jpg | True: 1   | Pred: 1   | Prob: 1.0000
IMG_20250920_114411742_HDR.jpg | True: 1   | Pred: 4   | Prob: 0.9999
IMG_20250920_114424698.jpg | True: 1   | Pred: 1   | Prob: 0.9992
IMG_20250920_114459816.jpg | True: 1   | Pred: 1   | Prob: 0.9930
IMG_20250920_114510244.jpg | True: 1   | Pred: 3   | Prob: 0.7214
IMG_20250920_114546151.jpg | True: 1   | Pred: 2   | Prob: 0.5447
IMG_20250920_114557923_HDR.jpg | True: 1   | Pred: 2   | Prob: 0.9989
IMG_20250920_114634292.jpg | True: 1   | Pred: 4   | Prob: 0.7257
IMG_20250920_114642676.jpg | True: 1   | Pred: 1   | Prob: 0.9947
IMG_20250920_114726232_HDR.jpg | True: 1   | Pred: 1   | Prob: 0.8158
IMG_20250920_114735995_HDR.jpg | True: 1   | Pred: 1   | Prob: 0.9537
IMG_20250

In [44]:
if hasattr(svm_model, "predict_proba"):
    svm_probs = svm_model.predict_proba(X_test)
else:
    scores = svm_model.decision_function(X_test)
    exp_scores = np.exp(scores - np.max(scores, axis=1, keepdims=True))
    svm_probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

svm_preds = np.argmax(svm_probs, axis=1)
svm_labels = getattr(svm_model, "classes_", log_labels)
svm_acc = (svm_labels[svm_preds] == y_test).mean()

print(f"\nSVM Accuracy on new images: {svm_acc:.3f}")
for i, name in enumerate(file_names):
    print(f"{name:<25} | True: {y_test[i]:<3} | Pred: {svm_labels[svm_preds[i]]:<3} | Prob: {svm_probs[i][svm_preds[i]]:.4f}")


SVM Accuracy on new images: 0.412
IMG_20250920_114308655.jpg | True: 1   | Pred: 1   | Prob: 0.8770
IMG_20250920_114321040.jpg | True: 1   | Pred: 1   | Prob: 0.8031
IMG_20250920_114337247.jpg | True: 1   | Pred: 1   | Prob: 0.7552
IMG_20250920_114349302.jpg | True: 1   | Pred: 1   | Prob: 0.8979
IMG_20250920_114411742_HDR.jpg | True: 1   | Pred: 3   | Prob: 0.5824
IMG_20250920_114424698.jpg | True: 1   | Pred: 1   | Prob: 0.5862
IMG_20250920_114459816.jpg | True: 1   | Pred: 1   | Prob: 0.6189
IMG_20250920_114510244.jpg | True: 1   | Pred: 2   | Prob: 0.5399
IMG_20250920_114546151.jpg | True: 1   | Pred: 1   | Prob: 0.6759
IMG_20250920_114557923_HDR.jpg | True: 1   | Pred: 1   | Prob: 0.6540
IMG_20250920_114634292.jpg | True: 1   | Pred: 3   | Prob: 0.5758
IMG_20250920_114642676.jpg | True: 1   | Pred: 1   | Prob: 0.6000
IMG_20250920_114726232_HDR.jpg | True: 1   | Pred: 1   | Prob: 0.5297
IMG_20250920_114735995_HDR.jpg | True: 1   | Pred: 1   | Prob: 0.7585
IMG_20250920_114810908.jp